<a href="https://colab.research.google.com/github/Huwa-hank/114-1PL/blob/main/week2_%E7%94%9F%E6%B4%BB%E6%B6%88%E8%B2%BB%E7%B4%80%E9%8C%84%E8%A1%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

目標：從 Sheet 讀「消費紀錄」→ 計總額/分類小計/AA 分攤 → 寫回 Sheet Summary 分頁。

AI 點子（可選）：請模型總結本週花錢習慣與建議（例如「外食過多」）。

Sheet 欄位：date, category, item, amount, payer

In [ ]:
!pip install gspread oauth2client gspread_dataframe

In [ ]:
import gspread
from google.auth import default
import pandas as pd
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.colab import auth
auth.authenticate_user()

from google.auth.transport.requests import Request
from google.auth import default

import datetime

# 🔑 如果用 Colab OAuth
creds, _ = default()
gc = gspread.authorize(creds)

# 開啟 Google Sheet（請改成你的表單名稱）
sheet = gc.open("日常財務紀錄表").sheet1

# 初始化欄位
columns = ["日期", "項目", "分類", "金額"]

def 輸入日期():
    while True:
        try:
            年 = int(input("請輸入年份 (例如 2025): "))
            月 = int(input("請輸入月份 (1-12): "))
            日 = int(input("請輸入日期 (1-31): "))

            # 嘗試建立日期物件，檢查是否有效
            日期物件 = datetime.date(年, 月, 日)
            return 日期物件.strftime("%Y-%m-%d")

        except ValueError:
            print("⚠️ 日期輸入錯誤，請重新輸入！")

def 輸入月份():
    while True:
        try:
            年 = int(input("請輸入年份 (例如 2025): "))
            月 = int(input("請輸入月份 (1-12): "))

            if 1 <= 月 <= 12:
                日期物件 = f"{年}-{月:02d}"  # 格式化成 YYYY-MM
                return 日期物件
            else:
                print("月份錯誤")

        except ValueError:
            print("⚠️ 日期輸入錯誤，請重新輸入！")

def 讀取紀錄():
    """從 Google Sheet 載入紀錄"""
    df = get_as_dataframe(sheet, evaluate_formulas=True, header=0)
    df = df.dropna(how="all")  # 移除空列
    if df.empty:
        df = pd.DataFrame(columns=columns)
    return df

def 寫入紀錄(df):
    """把 DataFrame 寫回 Google Sheet"""
    sheet.clear()
    set_with_dataframe(sheet, df)

def 新增紀錄():
    df = 讀取紀錄()
    日期 = 輸入日期()
    項目 = input("輸入項目: ")
    分類 = input("輸入分類: ")
    金額 = float(input("輸入金額: "))
    new_row = pd.DataFrame([[日期, 項目, 分類, 金額]], columns=columns)
    df = pd.concat([df, new_row], ignore_index=True)
    寫入紀錄(df)
    print("✅ 已新增紀錄！")

def 查看紀錄():
    df = 讀取紀錄()
    if df.empty:
        print("⚠️ 目前沒有紀錄")
    else:
        print(df)

def 月總額():
    df = 讀取紀錄()
    月份 = 輸入月份()
    month_data = df[df["日期"].str.startswith(月份)]
    total = month_data["金額"].sum()
    print(f"💰 {月份} 總額: {total}")

def 分類小計():
    df = 讀取紀錄()
    月份 = 輸入月份()
    month_data = df[df["日期"].str.startswith(月份)]
    if month_data.empty:
        print("⚠️ 該月份沒有紀錄")
    else:
        print("📊 分類小計：")
        print(month_data.groupby("分類")["金額"].sum())

# 🔹 主選單
def 主程式():
    while True:
        print("\n=== 消費紀錄系統 ===")
        print("1. 新增紀錄")
        print("2. 查看紀錄")
        print("3. 計算月總額")
        print("4. 計算分類小計")
        print("0. 離開")

        choice = input("請輸入選項: ")

        if choice == "1":
            新增紀錄()
        elif choice == "2":
            查看紀錄()
        elif choice == "3":
            月總額()
        elif choice == "4":
            分類小計()
        elif choice == "0":
            print("👋 程式結束")
            break
        else:
            print("⚠️ 無效選項，請重新輸入")

# 🚀 執行
主程式()
